In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import open3d as o3d
import copy

from utils import readXML, Intrinsics, Extrinsics, RGBD, crop_geometry, draw_registration_result, pick_points, manual_registration

ImportError: cannot import name 'crop_geometry' from 'utils' (C:\Users\lucas\Desktop\UPC\MIT\tactile2object\intrinsics_extrinsics\utils.py)

In [ ]:
xml_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/intrinsics_extrinsics/"

intrinsics = Intrinsics(xml_dir)
extrinsics = Extrinsics(xml_dir)
rgbds = RGBD(xml_dir, recording="20210709_004128")

# Stitching point clouds

### camera 020122061233

In [ ]:
camera = "020122061233"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_1 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)

#o3d.visualization.draw_geometries([pcd_1])

### camera 821312060044

In [ ]:
camera = "821312060044"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_2 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, np.linalg.inv(extrinsic))


### camera 020122061651

In [ ]:
camera = "020122061651"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_3 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, np.linalg.inv(extrinsic))


### camera 821312062243

In [ ]:
camera = "821312062243"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_4 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, np.linalg.inv(extrinsic))


### Stitched Point Cloud

In [ ]:
pcds = pcd_1 + pcd_2 + pcd_3 + pcd_4

o3d.visualization.draw_geometries([pcds])

# ICP

In [ ]:
obj_name = "005_tomato_soup_can"
obj_name = "003_cracker_box"

obj_path = f"C:/Users/lucas/Desktop/UPC/MIT/tactile2object/estimate_object_pose/ycb_converted/{obj_name}.ply"

obj_pcd = o3d.io.read_point_cloud(obj_path)
obj_mesh = o3d.io.read_triangle_mesh(obj_path)

obj_pcd.scale(0.001, center=(0,0,0))
obj_mesh.scale(0.001, center=(0,0,0))

In [ ]:
source = copy.deepcopy(obj_pcd)
target = copy.deepcopy(pcds)
transformation = manual_registration(source, target)

In [ ]:
obj_mesh_t = copy.deepcopy(obj_mesh).transform(transformation)

obj_mesh_t.compute_vertex_normals()
obj_mesh_t.paint_uniform_color([1, 0.706, 0])

o3d.visualization.draw_geometries([pcds, obj_mesh_t])